## Datapreprocessing <br>


In [59]:
from pathlib import Path
import tensorflow as tf
import functools
#params
params = {}
params['batch_size'] = 20
params['words'] = '../Data/words.txt'
params['tags'] = '../Data/tags.txt'
params['shuffle'] = False
params['batch_size'] = 2
params['prefech_size'] = 0
params['buffer_size'] = 5
params['epochs'] = 2
#sử dụng eager execution chúng ta không phải define một static graph
tf.enable_eager_execution()
   

### load data sử dụng tf.data dể feed data vào model <br>
Sử dụng tf.data để loadfile, sử dụng tf data là vì nó khắc phục tình trạng model bị đói data (data starving), thích hợp khi train sử dụng GPU <br> <br>

Ta sử dụng tf.data.Dataset.from_generator để load file, cho phép ta thực hiện một số bước tiền xử lý trước khi đưa vào model


Trích xuất character level cho mô hình cnn

In [72]:
def extract_character(token, default_value='<pad_char>'):
    # split token, tf.string_split trả lại dense vector
    split_char = tf.string_split(token, delimiter='')
    
    # convert sparse vector to dense vector
    split_char = tf.sparse.to_dense(split_char, default_value=default_value)
    
    return split_char

In [120]:
def generator_fn(words, tags):
    """
    input : đường dẫn words (từng dòng chứa token), tags(từng dòng chữa nhãn)
    """
    with Path(words).open('r') as f_words, Path(tags).open('r') as f_tags:
        for line_words, line_tags in zip(f_words, f_tags):
            # encode word, tag trước khi đưa vào model
            list_words = [w.encode() for w in line_words.strip().split()]
            list_tags = [t.encode() for t in line_tags.strip().split()]
            list_char_of_words = extract_character(list_words)
            # khi gọi hàm yeild sẽ trả về đối tượng của hàm này, và khi nào thực hiện
            # lặp nó sẽ sinh ra data khi gặp yeild và không dữ lại trên ram
            
            assert len(list_words) == len(list_tags)
            
            yield (list_words, len(list_words)), list_tags, list_char_of_words
            
gc = generator_fn(params['words'], params['tags'])
for i in gc:
    print(i)

(([b'I'], 1), [b'O'], <tf.Tensor: id=1560, shape=(1, 1), dtype=string, numpy=array([[b'I']], dtype=object)>)
(([b'I', b'live', b'in', b'San', b'Francisco'], 5), [b'O', b'O', b'O', b'B-LOC', b'I-LOC'], <tf.Tensor: id=1568, shape=(5, 9), dtype=string, numpy=
array([[b'I', b'<pad_char>', b'<pad_char>', b'<pad_char>', b'<pad_char>',
        b'<pad_char>', b'<pad_char>', b'<pad_char>', b'<pad_char>'],
       [b'l', b'i', b'v', b'e', b'<pad_char>', b'<pad_char>',
        b'<pad_char>', b'<pad_char>', b'<pad_char>'],
       [b'i', b'n', b'<pad_char>', b'<pad_char>', b'<pad_char>',
        b'<pad_char>', b'<pad_char>', b'<pad_char>', b'<pad_char>'],
       [b'S', b'a', b'n', b'<pad_char>', b'<pad_char>', b'<pad_char>',
        b'<pad_char>', b'<pad_char>', b'<pad_char>'],
       [b'F', b'r', b'a', b'n', b'c', b'i', b's', b'c', b'o']],
      dtype=object)>)
(([b'I', b'live'], 2), [b'O', b'O'], <tf.Tensor: id=1576, shape=(2, 4), dtype=string, numpy=
array([[b'I', b'<pad_char>', b'<pad_char>', b'

load word embeding

In [ ]:
def word_embeding():

contruct input data 

In [122]:
def input_fn(words, tags, shuffle = False):
    # định nghĩa kiểu data
    types = ((tf.string, tf.int32), tf.string)
    
    #định nghĩa ouput shape của data
    output_shape = (([None], ()), [None], tf.TensorShape([None]))
    
    # do ta phải pass một hàm vào và hàm generator_fn có tham số nên dể pass 
    # tham số ta dùng functools.partital
    dataset = tf.data.Dataset.from_generator(functools.partial(generator_fn, words, tags)
                                                       , types, \
                                             output_shape)
    
    # shuffle data, buffer_size ở đây là số phần tử tensor sẽ lấy vào buffer
    # và thực hiện shuffle, nếu data ở riêng từng lớp nên chọn buffer_size = số sample trong toàn data
    # sau dó thực hiện repeat lại suffle này cho các epochs khác
    if shuffle:
        dataset = dataset.shuffle(buffer_size = params['buffer_size']).repeat(count = params['epochs'])
    
    # giá trị padding thêm vào những câu ngắn sao cho chiều dài các câu bằng chiều
    # chiều dài câu lớn nhất trong batch
    default_value = (('<pad>', 0), 'O', ('<pad_char>'))
    
    # tạo batch
    #prefech là số batch mà CPU sẽ load sẵn lên ram cho GPU
    dataset = dataset.padded_batch(params['batch_size'], output_shape, default_value) \
    .prefetch(params['prefech_size'])
    
    return dataset

In [113]:
dataset = input_fn(params['words'], params['tags'])
iterator = dataset.make_one_shot_iterator()
node = iterator.get_next()
print(node)
node = iterator.get_next()
print (node)
with tf.Session() as sess:
    print(sess.run(node))

ValueError: The two structures don't have the same sequence length. Input structure has length 3, while shallow structure has length 2.

### CNNs - LSTM - CRF mode`

In [ ]:
CNN model cho character level embeding 

In [44]:
def f1(i):
    w = []
    w.append(i)
    return w

def createGenerator():
    mylist = range(3)
    for i in mylist:
        yield f1(i)
gc = createGenerator()
for i in gc:
    print(i)

[0]
[1]
[2]
